# Data, Databases, SQL, and interacting with databases using Python

For this exercise, we will be using data contained in the "homework" database on the Big Data for Social Science Class Server. This notebook will walk you through accessing the class data using IPython Notebook and while familiarizing you with the available class data.

## Table of Contents

- [Tables we will look at](#Tables-we-will-look-at)

    - [StarMetrics Database](#StarMetrics-Database)
    - [UMETRICS Grants Database](#UMETRICS-Grants-Database)
    - [USPTO Patents](#USPTO-Patents)

- [Databases and Python](#Databases-and-Python)

    - [Making and using a database connection](#Making-and-using-a-database-connection)
    - [Querying the database](#Querying-the-database)
    - [Close what you open](#Close-what-you-open)
    - [**Exercise 1 - `open_connection` function**](#Exercise-1---open_connection-function)
    
- [SQL](#SQL)

    - [Querying the database](#Querying-the-database)
    - [**Exercise 2**](#Exercise-2)
    - [WHERE clauses: Limiting the results](#WHERE-clauses:-Limiting-the-results)
    - [**Exercise 3**](#Exercise-3)
    - [JOIN: Connecting multiple tables](#JOIN:-Connecting-multiple-tables)
    - [GROUP BY and aggregate functions](#GROUP-BY-and-aggregate-functions)
    - [**Exercise 4**](#Exercise-4)
    - [ORDER BY](#ORDER-BY)
    - [Modifying the database](#Modifying-the-database)
    - [**Exercise 5**](#Exercise-5)

- [References](#References)

## Tables we will look at

- Back to the [Table of Contents](#Table-of-Contents)

For these exercises we will use tables in the homework database. These tables were created using data from the starmetrics, umetricsgrants and usptopatents databases. Each of these databases contain different types of information and are available for your use during this class. Also available to you, is your own database. These databases have the same name as your username. 

Quick description of the data available: 

### StarMetrics Database

- Back to the [Table of Contents](#Table-of-Contents)

The starmetrics database contains transactional data from universities that describe expenditures on federal research grants. The data includes four different types of expenditures: 

- 1) employee expenditures - this describes the people by occupation who charged time to federal grants.
- 2) vendor expenditures - this descirbes the businesses that goods were bought from federal grants.
- 3) subaward expenditures - this describes the universities and other institutions that are paid to collaborate from federal grants.
- 4) award expenditures - this describes the overhead that is associated with each federal grant.

### UMETRICS Grants Database 

- Back to the [Table of Contents](#Table-of-Contents)

The umetricsgrants database contains public data that describes NIH, NSF, USDA & NASA federal awards. This database was created by combining several small databases together to capture all the grant data in one database. The structure of the database tables are different depending on the source of the data.

### USPTO Patents

- Back to the [Table of Contents](#Table-of-Contents)

The USPTOPatents database contains public patent and inventor data. These data include all patents, inventors, assignees and their associated metadata on location, patent classes, etc. 

## Databases and Python

- Back to the [Table of Contents](#Table-of-Contents)

### Making and using a database connection

- Back to the [Table of Contents](#Table-of-Contents)

Python lets you interact with databases using SQL just like you would in any SQL gui or terminal. Python code can do SELECTs, CREATEs, INSERTs, UPDATEs, and DELETEs, and any other SQL, and the results are returned in a format that lets you interact with them after the SQL statements finish.

To interact with a database using python, first you have to connect to the database.

To create a database connection, you first must import that database's DB-API implementation, then you call the connect() function, passing it information on where to find the database to which you are trying to connect.

_Note: in most cases, it is best to place the values you pass to a function or method in variables:_

    # declare variables
    user = "username"
    password = "password"
    database = "homework"

    # invoke the connect() function, passing parameters in variables.
    db = MySQLdb.connect( user = user, passwd = password, db = database )

_rather than placing the values directly in the arguments in a function or method call:_

    # invoke the connect() function, passing values directly to function.
    db = MySQLdb.connect( user = "username", passwd = "password", db = "homework" )

_This allows you change how you populate the values (looking up a value from a command line parameter, for example, or the result of a database query) without also changing the line where the function is invoked.  If you separate setting values and using values, you make it easier to isolate problems with ** how you set the values** from problems with **how you use the values**._


In [ ]:
# imports
import MySQLdb

# declare variables - 
user = "<db_username>"
password = "<db_password>"
database = "homework"

# invoke the connect() function, passing parameters in variables.
db = MySQLdb.connect( user = user, passwd = password, db = database )

# output basic database connection info.
print( db )

Next, you use the connection to create a cursor. A cursor takes an SQL statement written as a string in python and passes it to the database, where it is executed. It then uses the results and converts them to a format that can be interacted with using python, and returns that transformed, usable response back to you.

To make a cursor, call the cursor() method on the connection object instance returned by the call to connect.

In [ ]:
# assumes you have alread opened the database using the connect() method.  If not:
#db = MySQLdb.connect( user = user, passwd = password, db = database )

# create mysql cursor that maps column names to values in the query result.
cursor = db.cursor( MySQLdb.cursors.DictCursor )

# output basic database cursor info.
print( cursor )

### Querying the database

- Back to the [Table of Contents](#Table-of-Contents)

To execute SQL, pass a string that contains an SQL statement to the cursor's execute() method, then call the fetchall() method to store the rows returned by the query in a list you can loop over.

For example: 

In [ ]:
# declare variables
sql_select = ""
result_count = -1
query_results = None
current_row = None

# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# run a simple select statement against our homework data.
#    When executing a SELECT, call to "execute" returns the
#    number of rows returned by the SELECT.
sql_select = "SELECT * FROM OSU_vendor LIMIT 10;"
result_count = cursor.execute( sql_select )

# how many rows returned?
print( "Found " + str( result_count ) + " rows" )

# get list of rows returned, so we can loop over them.
query_results = cursor.fetchall()

# loop over the results
for current_row in query_results:
        
    print( "==> " + current_row[ "university" ] + " - " + current_row[ "agency_text" ] + " - $" + str( current_row[ "paymentamount" ] ) + " - " + current_row[ "institutionid" ] )

#-- END loop over results --#

The way you create your cursor dictates the format of the rows you'll get back.

If you create your cursor by calling `cursor()` with no arguments, when you iterate over the results, each row will simply be a tuple (list) of the values in the row.  To reference a given value, you reference the position of that value in the row.

Example that shows the tuple row format:

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )

# close cursor if one already initialized - always close what you open.
if ( cursor is not None ):
    
    # open cursor - close it before opening another
    cursor.close()
    
#-- END check if cursor is initialized. --#

# create cursor with call to cursor(), no arguments.
cursor = db.cursor()

# run a simple select statement against our homework data.
sql_select = "SELECT * FROM OSU_vendor"
cursor.execute( sql_select )

# output the underlying structure of a single row.
single_row = cursor.fetchone()
print( single_row )

# get university - column 6 (7th column, starts with 0, so column 6)
university_value = single_row[ 6 ]
print( "====> university = " + university_value )

If you call the `cursor()` method with `MySQLdb.cursors.DictCursor` as the sole argument, when you access a row in the results from executing a query, each row is a dictionary that maps a column's name to that column's value, and you reference a given column by its name.  This is how our initial loop over rows worked above.

Example that shows the dictionary row format:

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )

# close cursor if one already initialized - always close what you open.
if ( cursor is not None ):
    
    # open cursor - close it before opening another
    cursor.close()
    
#-- END check if cursor is initialized. --#

# create cursor with call to cursor(), but asking it to make
#    a dictionary cursor (DictCursor).
cursor = db.cursor( MySQLdb.cursors.DictCursor )

# run a simple select statement against our homework data.
sql_select = "SELECT * FROM OSU_vendor"
cursor.execute( sql_select )

# output the underlying structure of a single row.
single_row = cursor.fetchone()
print( single_row )

# get university - who cares what position it is in!
university_value = single_row[ "university" ]
print( "====> university = " + university_value )

In general, we will be using the dictionary cursor, so opening our cursors with:

    cursor = db.cursor( MySQLdb.cursors.DictCursor )
    
rather than

    cursor = db.cursor()

Referencing items in a row by name means you can alter the order or contents of the results of a query and not have to change code that refers to those results unless you removed columns referenced in the code.  In addition, it makes one's code more self-documenting - the names of the columns you are retrieving are built right into the code that retrieves them.

<hr />

#### Fetch column names from result row

It also lets you retrieve the names of the columns present in a given query's results by grabbing the names (or keys) out of a given row's dictionary:

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
sql_select = ""
single_row = None
column_name_list = []

# select
sql_select = "SELECT * FROM OSU_vendor"
cursor.execute( sql_select )

# get a row, then get its keys
single_row = cursor.fetchone()
column_name_list = single_row.keys()

# what are my column names?
print( column_name_list )

<hr />

#### Fetch a certain number of rows

In the above example, we use `cursor.fetchone()` to retrieve a single row from the results of the query.  To retrieve one or more rows, but not necessarily all, use the `cursor.fetchmany()` method, passing it a parameter named `size` that contains the number of rows you want returned.

Example:

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
row_count = -1
desired_row_count = -1
result_list = []

# select
sql_select = "SELECT * FROM OSU_vendor"
row_count = cursor.execute( sql_select )

# total returned?
print( "Total rows = " + str( row_count ) )

# that is a lot!  Just get 10...
desired_row_count = 10
result_list = cursor.fetchmany( size = desired_row_count )

# loop over the results
for current_row in result_list:
        
    print( "==> " + current_row[ "university" ] + " - " + current_row[ "agency_text" ] + " - $" + str( current_row[ "paymentamount" ] ) + " - " + current_row[ "institutionid" ] )

#-- END loop over results --#

<hr />

#### Processing all rows in a large table

To pull the full table into memory, use `cursor.fetchall()`.

Be careful, this can take awhile depending on how big the table is, and if your table is truly "big", it might not be possible!

To see what we are getting into, we will use the SQL `COUNT()` function to see how many rows are in the table.  In order to access the `COUNT()` value in our dictionary result format, we'll assign this count a name using the `AS` SQL keyword.

Example:

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
row_count = -1
single_row = -1
vendor_count = -1

# select
sql_select = "SELECT COUNT( * ) AS 'vendor_count' FROM OSU_vendor"
row_count = cursor.execute( sql_select )

# get row and 'vendor_count'
single_row = cursor.fetchone()
vendor_count = single_row[ 'vendor_count' ]
print( "vendor_count = " + str( vendor_count ) )

That's kind of a lot.  But we should see what happens...  Load 'em anyway!

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
row_count = -1
single_row = -1
vendor_count = -1

# select
sql_select = "SELECT * FROM OSU_vendor"
row_count = cursor.execute( sql_select )

# default output
cursor.fetchall()

Jupyter eventually cuts off this statement, saving us from ourselves.

<hr />

#### Processing a large table one row at a time

If your data is too big to load all into memory at once, you can just read in one row at a time.  Example is below.  However, if you do this in a Jupyter notebook, you'll likely kill your browser - In the previous example, Jupyter detected that `cursor.fetchall()` was returning a lot of data and truncated that output before it could overwhelm the browser's ability to handle it.

_The example below pulls it in one row at a time, and so Jupyter won't know we need help.  70,000 rows might not sound like a lot, but it takes up enough space in browser memory to kill a browser.  I've hard-coded the number of rows below = 10.  Run it with all the rows at your own risk._

In [ ]:
# connect to database and make cursor if needed
#db = MySQLdb.connect( user = user, passwd = password, db = database )
#cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
row_count = -1
single_row = -1
row_counter = -1

# select
sql_select = "SELECT * FROM OSU_vendor"
cursor.execute( sql_select )

# you can also get rowcount from cursor
row_count = cursor.rowcount

# if we actually process all rows this way, it will kill browser.
#    Just do 10, to show how it works.
row_count = 10

# Use for loop to loop row_count times, fetching one row
#    each time through the loop.
for row_counter in range( row_count ):
        
    single_row = cursor.fetchone()
    print( "==> " + str( row_counter) + " - " + single_row )

#-- END loop over rows, fetching one at a time --#

<hr />

### Close what you open

- Back to the [Table of Contents](#Table-of-Contents)

Once you are done working with a database, you need to remember to always close any cursors you opened, and then close the database connection itself.

If you don't close what you open, you run the risk of consuming all of a database's available connections (there is always a limit to the number of concurrent connections you can have to a database) and locking yourself and others out of the database entirely until the connections time out.

To close cursors and the the database connection, call the `close()` method on either object.

Example:

In [ ]:
# close cursor
cursor.close()
    
# close connection
db.close()

### Exercise 1 - `open_connection` function

- Back to the [Table of Contents](#Table-of-Contents)

Our first exercise is an example of a pattern that will recur throughout the exercises for this class - we provide you a function definition, and you write the body of the function, then make use of it in subsequent exercises.

While future functions will likely be more complicated, our first exercise is straightforward - write the body of a function named `open_connection()` that accepts database connection information, creates a database connection using that information, then returns the connection to the user.

_Bonus: If any of the connection information is missing, should return `None`._

In [ ]:
# imports - in case we have not run any of the above cells this session.
import MySQLdb

def open_connection( username, password, database ):

    """
    Accept username, password, and database name.  Check to make
       sure there are values in each.  If so, create and return
       connection.  If not, return None.
    """
    
    # return reference
    connection_OUT = None
    
    ### BEGIN SOLUTION
    
    # check to see if all have values
    # username
    if ( ( username is not None ) and ( username != "" ) ):
    
        # password
        if ( ( password is not None ) and ( password != "" ) ):
            
            # database
            if ( ( database is not None ) and ( database != "" ) ):
        
                # got a value for everything.  Make connection.
                # invoke the connect() function, passing parameters in variables.
                connection_OUT = MySQLdb.connect( user = username, passwd = password, db = database )

            else:
                
                # no database.
                print( "In open_connection(): no database passed in, can't connect." )
                connection_OUT = None
            
            #-- END check see if database. --#
            
        else:
            
            # no password.
            print( "In open_connection(): no password passed in, can't connect." )
            connection_OUT = None
            
        #-- END check to see if password --#

    else:
            
        # no password.
        print( "In open_connection(): no password passed in, can't connect." )
        connection_OUT = None
        
    #-- END check to see if username --#
                
    ### END SOLUTION

    # return connection
    return connection_OUT
    
#-- END function open_connection() --#

To test your open_connection() method, first run the cell above that contains its function definition, so the function is created in memory and ready to use.

After the function definition has been run, use the code below to make a make a database connection using your function, create a cursor, then run a query.

In [ ]:
# declare variables - 
user = "<username>"
password = "<password>"
database = "homework"

# connect to database and make cursor
db = open_connection( user, password, database )
cursor = db.cursor( MySQLdb.cursors.DictCursor )

# declare variables
sql_select = ""
single_row = None
column_name_list = []

# select
sql_select = "SELECT * FROM OSU_vendor"
cursor.execute( sql_select )

# get a row, then get its keys
single_row = cursor.fetchone()
column_name_list = single_row.keys()

# what are my column names?
print( column_name_list )

In [ ]:
# imports
from nose.tools import assert_equal

# declare variables - set up your user here.
test_user = ""
test_password = ""
test_database = "homework"
test_db = None
test_cursor = None
test_select = ""
row_count = -1
single_row = None
vendor_count = -1

# set up database connection information
test_user = ""
test_password = ""
test_database = "homework"

# connect to database and make cursor
test_db = open_connection( test_user, test_password, test_database )

# is it the right type?
assert_equal( type( db ), MySQLdb.connections.Connection )

# create test cursor
test_cursor = test_db.cursor( MySQLdb.cursors.DictCursor )

# run test queries - get row count...
test_select = "SELECT * FROM OSU_vendor"
row_count = test_cursor.execute( test_select )

# ...then calculate 'vendor_count'
test_select = "SELECT COUNT( * ) AS 'vendor_count' FROM OSU_vendor"
test_cursor.execute( test_select )
single_row = test_cursor.fetchone()
vendor_count = single_row[ 'vendor_count' ]
#print( "vendor_count = " + str( vendor_count ) )

# Query results correct?
assert_equal( row_count, vendor_count )
assert_equal( 73081, vendor_count )

# close cursor and connection
test_cursor.close()
test_db.close()

## SQL

- Back to the [Table of Contents](#Table-of-Contents)

Now that we know how to make and use a connection to a database using Ipython, we can begin to master some SQL basics to help you get started with understanding the data and databases available. 

SQL is a quirky language. It is different from procedural languages like Python and is designed for a very specific purpose: to interact with relational data. It isn't structured like other languages, and while it can make data access easy, it also can make tasks that would be easy in other languages (though perhaps not exceptionally performant) confoundingly complex.  Let's dive in so you can see it for yourself!

### Querying the database

- Back to the [Table of Contents](#Table-of-Contents)

The basic method of querying the database is to use a select statement:

    SELECT *
    FROM OSU_vendor; 

Where:

- Columns or variables that would like returned are put in the SELECT clause (after the word "SELECT" but before the word "FROM").  An asterisk ( "\*" ) is a wildcard - it will return all columns for a given table.
- The name of the table (or names of the tables - more on this in a bit) you want to query is put after the word "FROM", in the FROM clause.
- It is considered good style to capitalize words in an SQL query that are SQL words, not variables, table names, or values you are filtering on or searching for, ie. SELECT, FROM, WHERE, etc.
- Although it isn't always necessary in MySQL, you should end SQL statements with a semi-colon.  It isn't required everywhere, but it is required in some contexts, so better to be aware and get into the habit.

Instead of specifying “all” columns ( "\*" ), you can specify which columns you want by name, in a comma-delimited list after "SELECT":

    SELECT uniqueawardnumber, fipscode, paymentamount
    FROM OSU_vendor;

You can specify calculations in the list of columns also:

    SELECT uniqueawardnumber, ( periodenddate - periodstartdate + 1 )
    FROM OSU_vendor;

And you can give those new columns names:

    SELECT uniqueawardnumber, ( periodenddate - periodstartdate + 1 ) AS num_days
    FROM OSU_vendor;
    
You can also use special keywords and functions in the SELECT clause.  For example, the keyword "DISTINCT", which only returns any given value in a given column once:

    SELECT DISTINCT uniqueawardnumber
    FROM OSU_vendor;
    
And "COUNT()", which returns a count of matching rows rather than a list:
    
    SELECT COUNT( DISTINCT uniqueawardnumber )
    FROM OSU_vendor;

### Exercise 2

- Back to the [Table of Contents](#Table-of-Contents)

In the code block below, store your database credentials, use them and your open_connection() method to create a connection to the "homework" database, then for each of the two questions:

- in the variable `select_#` (where `#` is the number of the question), store the SELECT statement you'll use to find an answer to the question.
- run the query and retrieve the answer to the question from the results of the query.
- in the variable `answer_#` (again where `#` is the number of the question), store the answer to the question.

In [ ]:
# Create connection to the "homework" database, then query the
#    "OSU_vendor" and "OSU_grant" tables to answer the following
#    questions: 

# Set database credentials here:
username = ""
password = ""
database = ""

# create connection and cursor.
connection = None
cursor = None

# ========================================================== #
# 1) Store the number of distinct vendors in the OSU vendor data in a column named "vendor_count" in the results of your SELECT.
# ========================================================== #

# SELECT text
select_1 = ""

# run select_!, then retrieve answer from results and store in answer_1

### BEGIN SOLUTION
select_1 = "SELECT COUNT( DISTINCT( 'fipscode' ) ) AS vendor_count FROM OSU_vendor;"
cursor.execute( select_1 )
row = cursor.fetchone()
vendor_count = row[ "vendor_count" ]
### END SOLUTION

# Store Answer 1
answer_1 = None

print( "Answer 1 = " + str( answer_1 ) + "; select_1 = " + select_1 )

# ========================================================== #
# 2) Store the number of distinct topics in the OSU topic data in a column named "topic_count" in the results of your SELECT.
# ========================================================== #

# SELECT text
select_2 = ""

### BEGIN SOLUTION
select_2 = "SELECT COUNT( DISTINCT( '?' ) ) AS topic_count FROM OSU_grant;"
cursor.execute( select_2 )
row = cursor.fetchone()
topic_count = row[ "topic_count" ]
### END SOLUTION

# Store answer_2
answer_2 = None

print( "Answer 2 = " + str( answer_2 ) + "; select_2 = " + select_2 )

# Close cursor, then connection:
cursor.close()
connection.close()

In [ ]:
# imports
from nose.tools import assert_equal

# declare variables - set up your user here.
#test_user = ""
#test_password = ""
#test_database = "homework"
test_db = None
test_cursor = None
test_select = ""
row_count = -1
single_row = None
test_select_1 = -1
vendor_count = -1
test_vendor_count = -1
test_select_2 = -1
topic_count = -1
test_topic_count = -1

# use user info from above?
#test_user = ""
#test_password = ""
#test_database = "homework"

# connect to database and make cursor
test_db = open_connection( test_user, test_password, test_database )
test_cursor = test_db.cursor( MySQLdb.cursors.DictCursor )

# run student query - select_1
row_count = test_cursor.execute( select_1 )
single_row = test_cursor.fetchone()
vendor_count = row[ "vendor_count" ]
assert_equal( answer_1, vendor_count )

# run test query - test_select_1
test_select_1 = ""
row_count = test_cursor.execute( test_select_1 )
single_row = test_cursor.fetchone()
test_vendor_count = row[ "vendor_count" ]
assert_equal( answer_1, test_vendor_count )

# run student query - select_2
row_count = test_cursor.execute( select_2 )
single_row = test_cursor.fetchone()
topic_count = row[ "topic_count" ]
assert_equal( answer_2, topic_count )

# run test query - select_2
test_select_2 = ""
row_count = test_cursor.execute( test_select_2 )
single_row = test_cursor.fetchone()
test_topic_count = row[ "topic_count" ]
assert_equal( answer_2, test_topic_count )

# close cursor and connection
test_cursor.close()
test_db.close()

### WHERE clauses: Limiting the results

- Back to the [Table of Contents](#Table-of-Contents)

In a SELECT query, you can add a WHERE clause to limit the results:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30';

Where:

- you are making conditional tests, just like in a Python "if" statement.
- EXCEPT here, instead of "==" being the equality operator, it is just "=".
- Comparison operators:

    - "**_`=`_**" - equal to
    - "**_`!=`_**" or "**_`<>`_**" - not equal to
    - "**_`<`_**" - less than
    - "**_`<=`_**" - less-than-or-equal-to
    - "**_`>`_**" - greater than
    - "**_`>=`_**" - greater-than-or-equal-to
    - "**_`LIKE`_**" and "**_`NOT LIKE`_**" - wild-card matching operator, where percent matches 0 or more characters ( "%" ) and an underscore matches any 1 character ( "_" ).
    - "**_`IN( value_list )`_**" and "**_`NOT IN( value_list )`_**" - checks whether the value to the left of the "IN", usually a column's value in a given row, is either IN or NOT IN the list on the right of the IN.
    
An example of using LIKE:

    SELECT *
    FROM OSU_vendor
    WHERE uniqueawardnumber LIKE '%EY022601%'

You can specify multiple conditions for matching in your WHERE clauses, as well, to more precisely filter the results of your query:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    
Note:

- when you are matching a column whose type is numeric, you just put the value in the query, with no quotation marks (just like in Python).
- when you are filtering a string column, you have to include the value you are looking for (the value on the right-hand side of the equal sign) in single-quotes. They must be single-quotes, too.  Unlike in Python, double-quotes have an entirely different meaning that single quotes in SQL, and can cause your query to fail.

Like "None" in Python, the signifier of an unset value in a column for a row is special - NULL.  To check for NULL, you use "IS NULL" or "IS NOT NULL", rather than the "=" or "!=".

    /* find missing values */
    SELECT *
    FROM OSU_vendor
    WHERE institutionid IS NULL;

You can also explicitly cut off the number of results your query returns using the LIMIT keyword.  Just LIMITing to 10 only returns the first 10 results for the query:

    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    LIMIT 10;
    
You can also use LIMIT to skip to the middle of the results by giving it two numbers, separated by a comma.  The first number is the number of records you want to skip, the second number is how many records you want to include after you skip:

    /* skip 10, the output 15 */
    SELECT *
    FROM OSU_vendor
    WHERE periodstartdate = '2014-06-30' and agency_abbrev = 'NSF'
    LIMIT 10, 15;

### Exercise 3

- Back to the [Table of Contents](#Table-of-Contents)

In the code block below, store your database credentials, use them and your open_connection() method to create a connection to the "homework" database, then for each of the two questions:

- in the variable `select_#` (where `#` is the number of the question), store the SELECT statement you'll use to find an answer to the question.
- run the query and retrieve the answer to the question from the results of the query.
- in the variable `answer_#` (again where `#` is the number of the question), store the answer to the question.

Once you have your code written, run it, and make sure the values you expect are printed out as output from the program.

In [ ]:
# Create connection to the "homework" database, then query the
#    "OSU_vendor" and "OSU_grant" tables to answer the following
#    questions: 

# Set database credentials here:
username = ""
password = ""
database = ""

# create connection and cursor.
connection = None
cursor = None

# ========================================================== #
# 3) What is the text that describes Topic 45?
# ========================================================== #

# SELECT text
select_3 = ""

# run select_!, then retrieve answer from results and store in answer_1

### BEGIN SOLUTION
select_3 = "?"
cursor.execute( select_1 )
row = cursor.fetchone()
vendor_count = row[ "?" ]
### END SOLUTION

# Store Answer 3
answer_3 = None

print( "Answer 3 = " + str( answer_3 ) + "; select_3 = " + select_3 )

# ========================================================== #
# 4) How well does topic ID 45 fit (percentage) to award number "1115005"?
# ========================================================== #

# SELECT text
select_4 = ""

### BEGIN SOLUTION
select_4 = "?"
cursor.execute( select_4 )
row = cursor.fetchone()
topic_count = row[ "topic_count" ]
### END SOLUTION

# Store answer_4
answer_4 = None

print( "Answer 4 = " + str( answer_4 ) + "; select_4 = " + select_4 )

# Close cursor, then connection:
cursor.close()
connection.close()

In [ ]:
# imports
from nose.tools import assert_equal

# declare variables - set up your user here.
#test_user = ""
#test_password = ""
#test_database = "homework"
test_db = None
test_cursor = None
test_select = ""
row_count = -1
single_row = None
topic_text = -1
test_topic_text = -1
agreement = -1
test_agreement = -1

# use user info from above?
#test_user = ""
#test_password = ""
#test_database = "homework"

# connect to database and make cursor
test_db = open_connection( test_user, test_password, test_database )
test_cursor = test_db.cursor( MySQLdb.cursors.DictCursor )

# run test queries - select_3
row_count = test_cursor.execute( select_3 )
single_row = test_cursor.fetchone()
topic_text = row[ "?" ]
assert_equal( answer_3, topic_text )

# run test query - test_select_3
test_select_3 = ""
row_count = test_cursor.execute( test_select_3 )
single_row = test_cursor.fetchone()
test_topic_text = row[ "?" ]
assert_equal( answer_3, test_topic_text )

# run test queries - select_4
row_count = test_cursor.execute( select_4 )
single_row = test_cursor.fetchone()
agreement = row[ "?" ]
assert_equal( answer_4, agreement )

# run test query - select_4
test_select_4 = ""
row_count = test_cursor.execute( test_select_4 )
single_row = test_cursor.fetchone()
test_agreement = row[ "?" ]
assert_equal( answer_4, test_agreement )

# close cursor and connection
test_cursor.close()
test_db.close()

### JOIN: Connecting multiple tables

- Back to the [Table of Contents](#Table-of-Contents)

We can specify multiple tables in the FROM clause of a select query. This is called a “join”. However, when we do, we need to remember to specify how to match up rows across the two tables. Usually, there is a column that is the same in both tables that can be used to match them up. For much of the starmetrics database, that would be a column like uniqueawardnumber or award_id. 

Also, we frequently give tables temporary short names to make it easy to refer to them.

    /* Lists the topics associated with each federal award */
    SELECT DISTINCT v.uniqueawardnumber, g.topic_text
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber;

We can still use regular WHERE clauses in these queries, too, to further filter:

    /* Lists the topics for each federal NSF award in 2012 */
    SELECT DISTINCT v.uniqueawardnumber, g.topic_text
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber
        AND agency_abbrev = "NSF"
        AND year(periodstartdate) = 2012;

Table joins are the most important feature of SQL databases; they are very powerful and allow us to create all kinds of complex queries. You can also join more than two tables if you like.

### GROUP BY and aggregate functions

- Back to the [Table of Contents](#Table-of-Contents)

Often, one thing that you want to do is to aggregate over multiple rows. For example, "What is the total expenditures for each award in 2012?" To do this, use a GROUP BY clause:

    /* sum vendor expenditures by award and filter by 2012 */
    SELECT uniqueawardnumber, SUM(paymentamount)
    FROM OSU_vendor
    WHERE year(periodstartdate) = 2012
    GROUP BY uniqueawardnumber;

There are a number of useful aggregate functions:

- **_SUM(column)_** : Calculate the sum of column for all the rows in each group
- **_AVG(column)_** : Calculate the numeric average for all of the rows in each group
- **_COUNT(column)_** : Count the number of rows in each group
- **_MIN(column) and MAX(column)_** : Find the minimum or maximum value of column in all the rows in each group

Often, it can be very powerful to combine GROUP BY and table joins. To figure out these queries, I recommend first getting the join to return the individual rows correctly, and then adding in the GROUP BY and aggregates.

### Exercise 4

- Back to the [Table of Contents](#Table-of-Contents)

In the code block below, store your database credentials, use them and your open_connection() method to create a connection to the "homework" database, then for each of the two questions:

- in the variable `select_#` (where `#` is the number of the question), store the SELECT statement you'll use to find an answer to the question.
- run the query and retrieve the answer to the question from the results of the query.
- in the variable `answer_#` (again where `#` is the number of the question), store the answer to the question.

Once you have your code written, run it, and make sure the values you expect are printed out as output from the program.

In [ ]:
# Create connection to the "homework" database, then query the
#    "OSU_vendor" and "OSU_grant" tables to answer the following
#    questions: 

# Set database credentials here:
username = ""
password = ""
database = ""

# create connection and cursor.
connection = None
cursor = None

# ========================================================== #
# 5) What are the total expenditures in 2012? 
# ========================================================== #

# SELECT text
select_5 = ""

# run select_5, then retrieve answer from results and store in answer_1

### BEGIN SOLUTION
select_5 = "?"
cursor.execute( select_5 )
row = cursor.fetchone()
vendor_count = row[ "?" ]
### END SOLUTION

# Store Answer 5
answer_5 = None

print( "Answer 5 = " + str( answer_5 ) + "; select_5 = " + select_5 )

# ========================================================== #
# 6) What are the total expenditures in 2012 on NSF grants? 
# ========================================================== #

# SELECT text
select_6 = ""

### BEGIN SOLUTION
select_6 = "?"
cursor.execute( select_6 )
row = cursor.fetchone()
topic_count = row[ "topic_count" ]
### END SOLUTION

# Store answer_6
answer_6 = None

print( "Answer 6 = " + str( answer_6 ) + "; select_4 = " + select_6 )

# ========================================================== #
# 7) What are the total expenditures in 2012 on NSF grants with topic ID 45?
# ========================================================== #

# SELECT text
select_7 = ""

### BEGIN SOLUTION
select_7 = "?"
cursor.execute( select_7 )
row = cursor.fetchone()
topic_count = row[ "topic_count" ]
### END SOLUTION

# Store answer_7
answer_7 = None

print( "Answer 7 = " + str( answer_7 ) + "; select_7 = " + select_7 )

# Close cursor, then connection:
cursor.close()
connection.close()

In [ ]:
# imports
from nose.tools import assert_equal

# declare variables - set up your user here.
#test_user = ""
#test_password = ""
#test_database = "homework"
test_db = None
test_cursor = None
test_select = ""
row_count = -1
single_row = None
q5_answer = -1
test_q5_answer = -1
q6_answer = -1
test_q6_answer = -1
q7_answer = -1
test_q7_answer = -1

# use user info from above?
#test_user = ""
#test_password = ""
#test_database = "homework"

# connect to database and make cursor
test_db = open_connection( test_user, test_password, test_database )
test_cursor = test_db.cursor( MySQLdb.cursors.DictCursor )

# run test queries - select_5
row_count = test_cursor.execute( select_5 )
single_row = test_cursor.fetchone()
q5_answer = row[ "?" ]
assert_equal( answer_5, q5_answer )

# run test query - test_select_5
test_select_5 = ""
row_count = test_cursor.execute( test_select_5 )
single_row = test_cursor.fetchone()
test_q5_answer = row[ "?" ]
assert_equal( answer_5, test_q5_answer )

# run test queries - select_6
row_count = test_cursor.execute( select_6 )
single_row = test_cursor.fetchone()
q6_answer = row[ "?" ]
assert_equal( answer_6, q6_answer )

# run test query - select_6
test_select_6 = ""
row_count = test_cursor.execute( test_select_6 )
single_row = test_cursor.fetchone()
test_q6_answer = row[ "?" ]
assert_equal( answer_6, test_q6_answer )

# run test queries - select_7
row_count = test_cursor.execute( select_7 )
single_row = test_cursor.fetchone()
q7_answer = row[ "?" ]
assert_equal( answer_7, q7_answer )

# run test query - select_7
test_select_7 = ""
row_count = test_cursor.execute( test_select_7 )
single_row = test_cursor.fetchone()
test_q7_answer = row[ "?" ]
assert_equal( answer_7, test_q7_answer )

# close cursor and connection
test_cursor.close()
test_db.close()

### ORDER BY

- Back to the [Table of Contents](#Table-of-Contents)

Normally, the results are returned in the order they appear in the database. However, it can be very useful to re-order the results using ORDER BY:

    SELECT uniqueawardnumber, paymentamount
    FROM OSU_vendor v, OSU_grant g
    WHERE v.uniqueawardnumber = g.uniqueawardnumber
        AND v.university = g.university
    ORDER BY g.award_id

(After you specify which column to order by, you can optionally specify either ASC for ascending order, or DESC for descending order.)

Using ORDER BY with custom column names can be really useful when combined with GROUP BY:

    SELECT uniqueawardnumber, SUM(paymentamount)
    FROM OSU_vendor
    WHERE year(periodstartdate) = 2012
    GROUP BY uniqueawardnumber
    ORDER BY sum(paymentamount) DESC

### Modifying the database

- Back to the [Table of Contents](#Table-of-Contents)

In addition to retrieving information from an existing database, you can also insert data into a database, update existing rows, and delete records using SQL.  Permissions on the homework, starmetrics, umetricsgrants and usptopatents databases will not allow you to modify the databases. For these exercises, we open an additional connection to your individual user database. 

Here are some example queries:

- **CREATE**: Adding a table to a database

        CREATE TABLE cjones.data (
        ID int(11) auto_increment primary key, 
        name_first varchar(20)
        name_last varchar(30))

- **INSERT**: Adding a row to a table

        INSERT INTO cjones.data
        (name_first, name_last)
        VALUES ('Christina', 'Jones')

- **UPDATE**: Changing data that is already in a table

        UPDATE cjones.data
        SET name_last = 'Johnson'
        WHERE name_first = 'Christina'
        
- **ALTER TABLE**: Changing the structure of an existing table

        ALTER TABLE cjones.data
            ADD COLUMN gender VARCHAR(1) DEFAULT 'F'

- **DELETE**: Removing one or more rows from a table

        DELETE FROM cjones.data
        WHERE name_last = 'johnson'

- **DELETE**: removing table from database

        DELETE cjones.data


Lastly, you can also CREATE a table using an existing table. 

- **CREATE**: Adding a table to a database

        CREATE TABLE cjones.osu_vendor (
        SELECT * FROM homework.OSU_vendor
        WHERE year(periodstartdate) = 2012
        and agency_abbrev = 'NSF');

### Exercise 4 

- Back to the [Table of Contents](#Table-of-Contents)

In [ ]:
#Create connection to the "homework" database "OSU_vendor" and "OSU_grant" tables and to your own database create and modify your own tables.


#Create Connection Here: 
user =
passwrd = 
db =
cur =

#1) Create a table in your home database with the following parameters - all vendor expenditures on 6-30-2014 with topic ID 45. Use name "OSU_exp_on_t45"

cur.execute()
list(cur.fetchall()) 

#Close Connection Here: 


## References 

- Back to the [Table of Contents](#Table-of-Contents)

The tables for this homework were created directly from the starmetrics and umetricsgrants databases. The following SQL code was used to generate the tables: 

In [ ]:
CREATE TABLE homework.osu_vendor (
    SELECT periodstartdate, periodenddate, v.uniqueawardnumber, recipientaccountnumber, institutionid, paymentamount, v.university, v.cfda, 
        v.zipcode, fipscode, statecode, countycode, c.agency, agency_abbrev, agency_text, sub_agency_text, program_title
    FROM starmetrics.vendor v
        LEFT JOIN starmetrics.zip_to_fip z on z.zipcode = v.zipcode
        LEFT JOIN starmetrics.cfda c on c.cfda = v.cfda
    WHERE v.university = "OSU"
        AND periodstartdate >= "2011-01-01" AND v.zipcode != "" )

In [ ]:
CREATE TABLE homework.OSU_grant (
    SELECT award_id, topic_id, model, application_id, proportion, seq, agency, topic_text, uniqueawardnumber, university 
    FROM umetricsgrants.topiclda t
        LEFT JOIN umetricsgrants.topiclda_text text using(topic_id, model)
        LEFT JOIN starmetrics.crosswalk c using(award_id) 
    WHERE t.model = "NSF"
        AND seq = 1
        AND university = "OSU")